# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
#pip install gmaps

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import vactionPy_apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
csv_path = "data/weather_df.csv"

weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
0,Bethel,41.3712,-73.4140,66.326,37,1,4.63
1,Kilindoni,-7.9139,39.6668,80.402,80,100,6.11
2,Tuktoyaktuk,69.4541,-133.0374,-9.400,77,75,3.60
3,Namatanai,-3.6667,152.4333,78.188,85,100,1.94
4,Kaminokawa,36.4333,139.9167,51.476,57,75,2.57
...,...,...,...,...,...,...,...
578,Mācherla,16.4833,79.4333,82.742,48,14,5.89
579,Gambēla,8.2500,34.5833,85.010,43,39,1.14
580,Arrondissement du Havre,49.5833,0.3333,40.874,81,90,2.57
581,Muyezerskiy,63.9333,31.6500,31.244,81,47,4.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
# Configure gmaps with API key
gmaps.configure(api_key=vactionPy_apiKey)

In [67]:
# Store 'Latitude' and 'Longitude' as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# 'Humidity' as float data type
humidity = weather_df["Humidity"].astype(float)

In [69]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
hotel_df = weather_df.loc[(weather_df['Temperature']>= 65)
                           & (weather_df['Temperature']< 80)
                            & (weather_df['Humidity']>= 10)
                           & (weather_df['Humidity']< 20)]
hotel_df.dropna(how='any')

hotel_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
387,Ocampo,21.6500,-101.5000,75.524,14,0,5.16
442,Calama,-22.4667,-68.9333,75.200,11,0,4.12
447,Najrān,17.4924,44.1277,77.000,15,0,2.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-60-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind (MPH),Hotel Name
2,2,Āsosa,10.0667,34.5333,74.552,18,44,0.31,
30,30,Riyadh,24.6877,46.7219,75.650,13,0,1.26,
257,257,Camargo,27.6667,-105.1667,74.156,16,62,4.38,
339,339,Pala,9.3642,14.9046,78.962,18,61,1.67,
399,399,Beni Suef,29.0731,31.0979,75.308,16,37,1.06,
454,454,Taoudenni,22.6783,-3.9836,77.126,15,1,3.37,
534,534,Gorom-Gorom,14.4439,-0.2361,79.340,10,17,4.65,
553,553,Yorosso,12.3536,-4.7761,79.826,12,97,2.15,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
